In [2]:
import tensorflow as tf
from IPython.display import Image
import numpy as np

## 1. 학습 모델 저장하고 재사용

In [4]:
data = np.loadtxt('../data.csv', delimiter=',', unpack=True, dtype='float32')
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [5]:
# 학습에 사용되지 않고 횟수만 카운트 하는 global 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2,W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [7]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables()) # 앞에 정의한 변수들을 가져옴

In [8]:
# model 디렉토리에 학습해둔 모델 불러오기(없으면 초기화)
ckpt = tf.train.get_checkpoint_state('../model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [43]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step : %d ' %sess.run(global_step),
          'Cost : %.3f ' %sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step : 1  Cost : 1.509 
Step : 2  Cost : 1.420 


In [44]:
saver.save(sess, '../model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-2'

In [45]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 : ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 : ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값 :  [0 1 1 0 0 0]
실제값 :  [0 1 2 0 0 2]
정확도 : 66.67


## 2. 텐서보드 사용하기

In [46]:
data = np.loadtxt('../data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [48]:
# 신경망 각 계층에 코드 붙이기
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimerzer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    
    train_op = optimizer.minimize(cost, global_step=global_step)

In [49]:
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost:0' shape=() dtype=string>

In [51]:
# 모델 load 및 초기화
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('../model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [56]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('../logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step : %d ' %sess.run(global_step),
          'Cost : %.3f ' %sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step : 1  Cost : 0.947 
Step : 2  Cost : 0.884 
Step : 3  Cost : 0.833 
Step : 4  Cost : 0.790 
Step : 5  Cost : 0.754 
Step : 6  Cost : 0.724 
Step : 7  Cost : 0.699 
Step : 8  Cost : 0.679 
Step : 9  Cost : 0.663 
Step : 10  Cost : 0.648 
Step : 11  Cost : 0.636 
Step : 12  Cost : 0.625 
Step : 13  Cost : 0.616 
Step : 14  Cost : 0.607 
Step : 15  Cost : 0.600 
Step : 16  Cost : 0.594 
Step : 17  Cost : 0.588 
Step : 18  Cost : 0.583 
Step : 19  Cost : 0.579 
Step : 20  Cost : 0.575 
Step : 21  Cost : 0.572 
Step : 22  Cost : 0.570 
Step : 23  Cost : 0.567 
Step : 24  Cost : 0.565 
Step : 25  Cost : 0.563 
Step : 26  Cost : 0.562 
Step : 27  Cost : 0.560 
Step : 28  Cost : 0.559 
Step : 29  Cost : 0.558 
Step : 30  Cost : 0.557 
Step : 31  Cost : 0.557 
Step : 32  Cost : 0.556 
Step : 33  Cost : 0.555 
Step : 34  Cost : 0.555 
Step : 35  Cost : 0.554 
Step : 36  Cost : 0.554 
Step : 37  Cost : 0.554 
Step : 38  Cost : 0.553 
Step : 39  Cost : 0.553 
Step : 40  Cost : 0.553 
Step : 41

In [57]:
summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

In [58]:
saver.save(sess, '../model/dnn.ckpt', global_step=global_step)
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 : ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값 : ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 : %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값 :  [0 1 2 0 0 2]
실제값 :  [0 1 2 0 0 2]
정확도 : 100.00
